In [1]:
#GOAL: Make edits to HMM class here, then push to hmm.py
import numpy as np
import hmm

In [2]:
robot = hmm.robot_stats()

In [5]:
# robot.emis_matrix_list[0]
# robot.I_dict

In [10]:
class HMM(object):
    ''' Simple Hidden Markov Model implementation.  User provides
        transition, emission and initial probabilities in dictionaries
        mapping 2-character codes onto floating-point probabilities
        for those table entries.  States and emissions are represented
        with single characters.  Emission symbols comes from a finite.  '''
    
    def __init__(self, A, E, I, emis_matrix_list):
        ''' Initialize the HMM given transition, emission and initial
            probability tables. '''
        
        # put state labels to the set self.Q
        self.Q, self.S = set(), set() # states and symbols
        for a, prob in A.items():
#             asrc, adst = a[0], a[1]
            asrc, adst = a.split('|')
#             print('asrc: ', asrc)
#             print('adst: ', adst)
            self.Q.add(asrc)
            self.Q.add(adst)
            
        # add all the symbols to the set self.S
        for e, prob in E.items():
#             eq, es = e[0], e[1]
            eq, es = e.split('|')
            self.Q.add(eq)
            self.S.add(es)
        
        self.Q = sorted(list(self.Q))
        self.S = sorted(list(self.S))
        
        # create maps from state labels / emission symbols to integers
        # that function as unique IDs
        qmap, smap = {}, {}
        for i in range(len(self.Q)): qmap[self.Q[i]] = i
        for i in range(len(self.S)): smap[self.S[i]] = i
        lenq = len(self.Q)
        
        # create and populate transition probability matrix
        self.A = np.zeros(shape=(lenq, lenq), dtype=float)
        for a, prob in A.items():
#             asrc, adst = a[0], a[1]
            asrc, adst = a.split('|')
            self.A[qmap[asrc], qmap[adst]] = prob
        # make A stochastic (i.e. make rows add to 1)
        self.A /= self.A.sum(axis=1)[:, np.newaxis]
        
        # create and populate emission probability matrix
        self.E = emis_matrix_list[0]
        
        # initial probabilities
        self.I = [ 0.0 ] * len(self.Q)
        for a, prob in I.items():
            self.I[qmap[a]] = prob
        # make I stochastic (i.e. adds to 1)
        self.I = np.divide(self.I, sum(self.I))
#         self.I = I
        
        self.qmap, self.smap = qmap, smap
        
        #Accept emis_matrix_list
        self.emis_matrix_list = emis_matrix_list
        
    def viterbi(self, x):
        ''' Given sequence of emissions, return the most probable path
            along with its probability. '''
        
        #TODO: Use different smap for each observation
        
        #TO-DO: Change to consider lists of possible cells, per observation
        x = list(map(self.smap.get, x)) # turn emission characters into ids
        print('smap: ', self.smap)
        print('smap x: ', x)
        nrow, ncol = len(self.Q), len(x) #len trans_matrix = 87, len obs = 11
        mat   = np.zeros(shape=(nrow, ncol), dtype=float) # prob
        matTb = np.zeros(shape=(nrow, ncol), dtype=int)   # backtrace
        
        # Fill in first column
        for i in range(0, nrow):
            mat[i, 0] = self.E[i, x[0]] * self.I[i] #TODO: Confirm E
        
#         for i in range(0, nrow): #nrow = 87
#             mat[i, 0] = self.emis_matrix_list[0][i, x[0]] * self.I[i] #Fix x
            
        # Fill in rest of prob and Tb tables
        for j in range(1, ncol):
            for i in range(0, nrow):
                ep = self.E[i, x[j]] #TODO: Confirm E
                mx, mxi = mat[0, j-1] * self.A[0, i] * ep, 0
                for i2 in range(1, nrow):
                    pr = mat[i2, j-1] * self.A[i2, i] * ep
                    if pr > mx:
                        mx, mxi = pr, i2
                mat[i, j], matTb[i, j] = mx, mxi
                
        # Find final state with maximal probability
        omx, omxi = mat[0, ncol-1], 0
        for i in range(1, nrow):
            if mat[i, ncol-1] > omx:
                omx, omxi = mat[i, ncol-1], i
                
        # Backtrace
        i, p = omxi, [omxi]
        for j in range(ncol-1, 0, -1):
            i = matTb[i, j]
            p.append(i)
        p = ''.join(map(lambda x: self.Q[x], p[::-1]))
        return omx, p # Return probability and path

In [11]:
hmm = HMM({"F|F":0.9, "F|L":0.1, "L|F":0.1, "L|L":0.9}, # transition matrix A
          {"F|H":0.5, "F|T":0.4, "F|M": 0.1, "L|H":0.75, "L|T":0.15, "F|M": 0.1}, # emission matrix E
          {"F":0.5, "L":0.5},
         robot.emis_matrix_list) # initial probabilities I
hmm.I


array([0.5, 0.5])

In [19]:
hmm2 = HMM(robot.T, #transition matrix A
           robot.E_list[0], #DELETE THIS LATER
           robot.I_dict, #initial probabilities I
           robot.emis_matrix_list) # emission matrices E
# hmm2.A.shape
hmm2.E.shape

(10, 87)

In [13]:
jprobOpt, path = hmm.viterbi("HMTHMT")

smap:  {'H': 0, 'M': 1, 'T': 2}
smap x:  [0, 1, 2, 0, 1, 2]


In [15]:
# jprobOpt, path = hmm2.viterbi("HMTHMT")

In [ ]:
#GOAL: Chang